Import

In [2]:
import subprocess
import shlex
import os.path
import sys
import pickle
import datetime
import tweepy
import matplotlib.pyplot as plt 
import numpy as np
from scipy import spatial  
from sklearn.metrics import r2_score
import string
import numpy as np
import matplotlib.pyplot as plt
import collections
from itertools import tee, islice
from sklearn.decomposition import PCA
import math

Other Helper Functions

In [3]:
def get_list_of_users(tweets):

    list_of_user = []

    for tweet in tweets:
        user_id = tweet.user.id
        if user_id not in list_of_user:
            list_of_user.append(user_id)

    return list_of_user

Import Tweet and Related Data

In [ ]:
# Where you want to save the coverted files # TO BE REMOVED
test_directory = "" 

In [4]:
#Location of all the ".p" files containing tweets of friends
directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/" 

# Location of sentiment analysis file
sentiment_directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/Sentiment/"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"

In [5]:
# Tweets output from the function, get_inital_tweets
initial_tweet_file = directory + 'user_tweet_large.p'
initial_tweets = pickle.load(open(initial_tweet_file, "rb"))
trending_tweet_file = directory + 'trending_tweets.p'
trending_tweets = pickle.load(open(trending_tweet_file, "rb"))

In [6]:
print(len(trending_tweets))
list_of_users = get_list_of_users(trending_tweets)

740


In [3]:
# Don't use this if using above
already_analyzed = pickle.load(open(analyzed_users, "rb"))
list_of_users = []

for uid in already_analyzed:
    list_of_users.append(uid)

SVM - Experimental

In [7]:
sentiment_directory = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/Sentiment/"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"
analyzed_users = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Python Tweets/data_analyzed_1.p"


data_point_n_4 = []
data_point_n_3 = []
data_point_n_2 = []
data_point_n_1 = []
data_point_n_0 = []
data_point_p_1 = []
data_point_p_2 = []
data_point_p_3 = []
data_point_p_4 = []

already_analyzed = pickle.load(open(analyzed_users, "rb"))
list_of_users = []

for uid in already_analyzed:
    list_of_users.append(uid)

# Analyze all the  users in initial_tweets
for user in list_of_users:
    
    file_name = sentiment_directory + str(user) + '_sent.p'
    
    if os.path.exists(file_name):
        
        tweet_sentiment = pickle.load(open(file_name, "rb"))
        
        for tweet_id in tweet_sentiment:
            positive = 0
            negative = 0
            neutral = 0
            contain_pos = 0
            contain_neg = 0
            
            tweet_sent = tweet_sentiment[tweet_id]['combined']

            if 'friend' in tweet_sentiment[tweet_id]:
                for tweet in tweet_sentiment[tweet_id]['friend']:
                    friend_sentiment = tweet_sentiment[tweet_id]['friend'][tweet]['combined']
                    
                    if friend_sentiment > 0:
                        positive += 1
                    elif friend_sentiment < 0:
                        negative += 1
                    else:
                        neutral += 1
                        
                    friend_sentiment_pos = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][0]
                    friend_sentiment_neg = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][1]
                    
                    if friend_sentiment_pos > 1:
                        contain_pos += 1
                        
                    if friend_sentiment_neg < -1:
                        contain_neg += 1
                            
                total = positive + negative + neutral
                
                if total > 0:
                    positive_percent = positive/total
                    negative_percent = negative/total
                    neutral_percent = neutral/total
                    
                    cont_pos_percent = contain_pos/total
                    cont_neg_percent = contain_neg/total
                    
                    temp = [positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent] #[cont_pos_percent, cont_neg_percent, neutral_percent]#[positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent]
                    temp = np.asarray(temp)

                    if tweet_sent == -4:
                        data_point_n_4.append(temp)
                    elif tweet_sent == -3:
                        data_point_n_3.append(temp)
                    elif tweet_sent == -2:
                        data_point_n_2.append(temp)
                    elif tweet_sent == -1:
                        data_point_n_1.append(temp)
                    elif tweet_sent == 0:
                        data_point_n_0.append(temp)
                    elif tweet_sent == 1:
                        data_point_p_1.append(temp)
                    elif tweet_sent == 2:
                        data_point_p_2.append(temp)
                    elif tweet_sent == 3:
                        data_point_p_3.append(temp)
                    elif tweet_sent == 4:
                        data_point_p_4.append(temp)

data_point_n_4 = np.asarray(data_point_n_4)
data_point_n_3 = np.asarray(data_point_n_3)
data_point_n_2 = np.asarray(data_point_n_2)
data_point_n_1 = np.asarray(data_point_n_1)
data_point_n_0 = np.asarray(data_point_n_0)
data_point_p_1 = np.asarray(data_point_p_1)
data_point_p_2 = np.asarray(data_point_p_2)
data_point_p_3 = np.asarray(data_point_p_3)
data_point_p_4 = np.asarray(data_point_p_4)

In [8]:
from sklearn import svm
import random

resample_index = 0

data_point_n_4 = data_point_n_4.tolist()
data_point_n_3 = data_point_n_3.tolist()
data_point_n_2 = data_point_n_2.tolist()
data_point_n_1 = data_point_n_1.tolist()
data_point_n_0 = data_point_n_0.tolist()
data_point_p_1 = data_point_p_1.tolist()
data_point_p_2 = data_point_p_2.tolist()
data_point_p_3 = data_point_p_3.tolist()
data_point_p_4 = data_point_p_4.tolist()

n_4_label = np.zeros(len(data_point_n_4))
n_3_label = np.zeros(len(data_point_n_3))
n_2_label = np.zeros(len(data_point_n_2))
n_1_label = np.zeros(len(data_point_n_1))
n_0_label = np.zeros(len(data_point_n_0))
p_1_label = np.zeros(len(data_point_p_1))
p_2_label = np.zeros(len(data_point_p_2))
p_3_label = np.zeros(len(data_point_p_3))
p_4_label = np.zeros(len(data_point_p_4))

max_length = max(len(n_4_label), len(n_3_label), len(n_2_label), len(n_1_label), len(n_0_label), len(p_1_label), len(p_2_label), len(p_3_label), len(p_4_label))

for i in range(max_length):
    try:
        n_4_label[i] = -4
    except:
        a = 1
    try:
        n_3_label[i] = -3
    except:
        a = 1
    try:
        n_2_label[i] = -2
    except:
        a = 1
    try:
        n_1_label[i] = -1
    except:
        a = 1
    try:
        p_1_label[i] = 1
    except:
        a = 1
    try:
        p_2_label[i] = 2
    except:
        a = 1
    try:
        p_3_label[i] = 3
    except:
        a = 1
    try:
        p_4_label[i] = 4
    except:
        a = 1
        
n_4_label = n_4_label.tolist()
n_3_label = n_3_label.tolist()
n_2_label = n_2_label.tolist()
n_1_label = n_1_label.tolist()
n_0_label = n_0_label.tolist()
p_1_label = p_1_label.tolist()
p_2_label = p_2_label.tolist()
p_3_label = p_3_label.tolist()
p_4_label = p_4_label.tolist()

''' 
data_point_n_4 = data_point_n_4.tolist()
data_point_n_3 = data_point_n_3.tolist()
data_point_n_2 = data_point_n_2.tolist()
data_point_n_1 = data_point_n_1.tolist()
data_point_n_0 = data_point_n_0.tolist()
data_point_p_1 = data_point_p_1.tolist()
data_point_p_2 = data_point_p_2.tolist()
data_point_p_3 = data_point_p_3.tolist()
data_point_p_4 = data_point_p_4.tolist()
'''


all_labels = n_4_label + n_3_label + n_2_label + n_1_label + n_0_label + p_1_label + p_2_label + p_3_label + p_4_label
all_labels = np.asarray(all_labels)

all_points = data_point_n_4 + data_point_n_3 + data_point_n_2 + data_point_n_1 + data_point_n_0 + data_point_p_1 + data_point_p_2 + data_point_p_3 + data_point_p_4
all_points = np.asarray(all_points)
print(all_points)

clf = svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo') #svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo')  #svm.SVC(kernel='poly', degree=4, C=1, decision_function_shape='ovo') #svm.SVC(kernel='linear', C=1, decision_function_shape='ovo') #svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo') #svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo') #svm.SVC(kernel='poly', degree=3, C=1, decision_function_shape='ovo') #svm.NuSVC(gamma='auto') #kernel='poly', probability=True
clf.fit(all_points, all_labels)
#clf.fit(np.asarray(data_point_p_4), np.asarray(p_4_label), sample_weight=20)
#tech = clf.predict([[0.2, 0.3, 0.5, 0.5, 0.5]])
#print(int(tech[0]))

[[0.33333333 0.1468254  0.51984127 0.         1.        ]
 [0.41666667 0.         0.58333333 0.         1.        ]
 [0.66666667 0.         0.33333333 0.         1.        ]
 ...
 [0.         0.         1.         1.         0.        ]
 [0.33333333 0.         0.66666667 1.         0.        ]
 [0.24847251 0.13849287 0.61303462 1.         0.        ]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [9]:
tech = clf.predict([[0.24847251, 0.13849287, 0.61303462, 1,0]])
print(int(tech[0]))
print(all_labels)

1
[-4. -4. -4. ...  4.  4.  4.]


In [9]:
# Prep for Testing Model
result = {}

NZ_tweet_dir = predictive_model = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Partner Code/"
predictive_model = "L:/Users/Daniel/Documents/2020-2021/2021 Winter Courses/COG403/Assignments/Project/Partner Code/Sentiment/"

#tweet_file_name = predictive_model  + 'user_tweet_large_NZ_1.p'
#predictive_user_tweets = pickle.load(open(tweet_file_name, "rb"))

trending_tweet_file_NZ = NZ_tweet_dir + 'trending_tweets_NZ.p'
trending_tweets_NZ = pickle.load(open(trending_tweet_file_NZ, "rb"))

list_of_users_NZ = get_list_of_users(trending_tweets_NZ)

analyzed_users_NZ = predictive_model  + 'data_analyzed_1.p'
already_analyzed_NZ = pickle.load(open(analyzed_users_NZ, "rb"))

for uid in already_analyzed_NZ:
    if uid not in list_of_users_NZ:
        list_of_users_NZ.append(uid)
        
print(len(list_of_users_NZ))

191


In [10]:
# SVM

total_count = 0
accurate = 0

n_4_total = 0
n_4_accurate = 0
n_4_miss = 0
n_4_miss_list = []

n_3_total = 0
n_3_accurate = 0
n_3_miss = 0
n_3_miss_list = []

n_2_total = 0
n_2_accurate = 0
n_2_miss = 0
n_2_miss_list = []

n_1_total = 0
n_1_accurate = 0
n_1_miss = 0
n_1_miss_list = []

n_0_total = 0
n_0_accurate = 0
n_0_miss = 0
n_0_miss_list = []

p_1_total = 0
p_1_accurate = 0
p_1_miss = 0
p_1_miss_list = []

p_2_total = 0
p_2_accurate = 0
p_2_miss = 0
p_2_miss_list = []

p_3_total = 0
p_3_accurate = 0
p_3_miss = 0
p_3_miss_list = []

p_4_total = 0
p_4_accurate = 0
p_4_miss = 0
p_4_miss_list = []
    
# Analyze all the  users in initial_tweets
for user in list_of_users_NZ:
    
    file_name = predictive_model + str(user) + '_sent.p'
    
    if os.path.exists(file_name):
        
        tweet_sentiment = pickle.load(open(file_name, "rb"))
        
        for tweet_id in tweet_sentiment:

            positive = 0
            negative = 0
            neutral = 0
            contain_pos = 0
            contain_neg = 0
            
            tweet_sent = tweet_sentiment[tweet_id]['combined']
            
            if tweet_sent != 100:

                if 'friend' in tweet_sentiment[tweet_id]:
                    for tweet in tweet_sentiment[tweet_id]['friend']:

                        friend_sentiment = tweet_sentiment[tweet_id]['friend'][tweet]['combined']
                        if friend_sentiment > 0:
                            positive += 1
                        elif friend_sentiment < 0:
                            negative += 1
                        else:
                            neutral += 1
                            
                        friend_sentiment_pos = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][0]
                        friend_sentiment_neg = tweet_sentiment[tweet_id]['friend'][tweet]['sentiment'][1]
                    
                        if friend_sentiment_pos > 1:
                            contain_pos += 1
                        
                        if friend_sentiment_neg < -1:
                            contain_neg += 1
                            

                total = positive + negative + neutral
                
                if total > 0:
                    positve_percent = positive/total
                    negative_percent = negative/total
                    neutral_percent = neutral/total
                    
                    cont_pos_percent = contain_pos/total
                    cont_neg_percent = contain_neg/total
                    
                    temp = [positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent] #[cont_pos_percent, cont_neg_percent, neutral_percent]#[positive_percent, negative_percent, neutral_percent, cont_pos_percent, cont_neg_percent]
                    temp = np.asarray(temp)

                    exemplar = int(clf.predict([temp])[0])
                    total_count += 1
                    
                    if tweet_sent == exemplar:
                        accurate += 1

In [11]:
print("Accurate " + str(accurate))
print("Total Count " + str(total_count))
print("% Accurate " + str(accurate/total_count))


Accurate 4066
Total Count 5164
% Accurate 0.7873741285824942
